In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
batch_size = 64  # Batch size for training.
epochs = 80  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "zh_language_cleansed2.txt"

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

# pretty sure this is building one hot vector representation of the data
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 1000
Number of unique input tokens: 53
Number of unique output tokens: 1390
Max sequence length for inputs: 21
Max sequence length for outputs: 25


In [4]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("chinese_model")

Train on 800 samples, validate on 200 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/80
800/800 [==============================] - 13s 16ms/sample - loss: 3.1408 - acc: 0.6983 - val_loss: 1.6270 - val_acc: 0.7542
Epoch 2/80
800/800 [==============================] - 10s 12ms/sample - loss: 1.6137 - acc: 0.7584 - val_loss: 1.6386 - val_acc: 0.7542
Epoch 3/80
800/800 [==============================] - 9s 11ms/sample - loss: 1.5955 - acc: 0.7613 - val_loss: 1.6310 - val_acc: 0.7684
Epoch 4/80
800/800 [==============================] - 9s 11ms/sample - loss: 1.5577 - acc: 0.7624 - val_loss: 1.6541 - val_acc: 0.7632
Epoch 5/80
800/800 [==============================] - 9s 11ms/sample - loss: 1.5579 - acc: 0.7607 - val_loss: 1.6581 - val_acc: 0.7688
Epoch 6/80
800/800 [==============================] - 9s 12ms/sample - loss: 1.5617 - acc: 0.7610 - val_loss: 1.6624 - val_acc: 0.7602
Epoch 7/80
800/800 [============================

800/800 [==============================] - 9s 11ms/sample - loss: 0.8277 - acc: 0.8198 - val_loss: 1.5067 - val_acc: 0.7938
Epoch 59/80
800/800 [==============================] - 9s 11ms/sample - loss: 0.8193 - acc: 0.8217 - val_loss: 1.5049 - val_acc: 0.7890
Epoch 60/80
800/800 [==============================] - 9s 11ms/sample - loss: 0.8043 - acc: 0.8250 - val_loss: 1.4975 - val_acc: 0.7922
Epoch 61/80
800/800 [==============================] - 9s 11ms/sample - loss: 0.7925 - acc: 0.8263 - val_loss: 1.5066 - val_acc: 0.7928
Epoch 62/80
800/800 [==============================] - 10s 12ms/sample - loss: 0.7795 - acc: 0.8279 - val_loss: 1.5046 - val_acc: 0.7882
Epoch 63/80
800/800 [==============================] - 10s 12ms/sample - loss: 0.7664 - acc: 0.8306 - val_loss: 1.4959 - val_acc: 0.7956
Epoch 64/80
800/800 [==============================] - 10s 12ms/sample - loss: 0.7653 - acc: 0.8329 - val_loss: 1.4944 - val_acc: 0.7968
Epoch 65/80
800/800 [==============================] - 9s

In [6]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("chinese_model")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
test_text = "huixuanqu"
one_hot_vector = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
for t, char in enumerate(test_text):
    one_hot_vector[0, t, input_token_index[char]] = 1.0
one_hot_vector[0, t + 1 :, input_token_index[" "]] = 1.0

In [11]:
decode_sequence(one_hot_vector[0:1])

'苏凯佳\n'

In [9]:
for seq_index in range(20):
    print("-")
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
#     print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)



-
Input sentence: X XZ WN B
Decoded sentence: 蜜桃

-
Input sentence: reiu 
Decoded sentence: A AN

-
Input sentence:  iyumeko
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: A KA AA C
Decoded sentence: A AA N

-
Input sentence: Atticus
Decoded sentence: A AA N A  E  E N N

-
Input sentence: eryu
Decoded sentence: Been

-
Input sentence: f RM QSgo Ov
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: h OE Pv BYmut N
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: forevertjt
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: X XZ WN B
Decoded sentence: 蜜桃

-
Input sentence: Ninja Zyd
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: L Uvjn Uatbvgc
Decoded sentence: 水秋寒

-
Input sentence: Kinky Horny SM
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: Bwl XYu BKdi QI
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: Hys Thomas
Decoded sentence: 夜夜教主

-
Input sentence: Xiaomeng mimi
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: O Ffuty UU Rv
Decoded sentence: 小狐狸不熬夜🌙✨⭐️

-
Input sentence: Asifmehr
D